####  Importing Random Module

In [ ]:
import random

#### Defining Responses

In [ ]:
responses = {
    "greeting": ["Hello! How can I help you today?", "Hi there! What can I do for you?", "Greetings! How may I assist you?"],
    "help": ["I'm here to help! What do you need assistance with?", "Sure, let me know how I can assist you.", "Feel free to ask me any question."],
    "default": ["I'm sorry, I didn't understand that. Can you please rephrase?", "Could you clarify your query? I'm here to help!"],
    "thanks": ["You're welcome! Let me know if you need anything else.", "Glad I could help!", "Anytime!"],
}

- A dictionary named `responses` is created to store different categories of responses that the chatbot will use based on the user's input. There are four categories:
- `greeting`: Responses when the user greets the bot (e.g., "Hello", "Hi").
- `help`: Responses when the user asks for help.
- `default`: Responses for unrecognized input, used if the user's query doesn't match any of the other categories.
- `thanks`: Responses when the user expresses gratitude (e.g., "Thank you").

- The `random.choice()` function will later be used to select a random response from each list when a specific category is triggered.

#### Defining the `chatbot_response` Function

In [ ]:
def chatbot_response(user_input):
    user_input = user_input.lower()
    if "hello" in user_input or "hi" in user_input or "hey" in user_input:
        return random.choice(responses["greeting"])
    elif "help" in user_input or "assist" in user_input:
        return random.choice(responses["help"])
    elif "thank" in user_input:
        return random.choice(responses["thanks"])
    else:
        return random.choice(responses["default"])

- This function processes the user’s input and returns an appropriate response based on the content of the input.
    - The input is converted to lowercase (user_input = user_input.lower()) to make the matching case-insensitive, ensuring that the bot can understand the input regardless of whether the user uses uppercase or lowercase letters.
    - The function checks for certain keywords in the input:
- `Greeting`: If the input contains "hello", "hi", or "hey", it returns a random greeting response.
- `Help`: If the input contains "help" or "assist", the chatbot will offer assistance.
- `Thanks`: If the input contains "thank", the chatbot will respond with a gratitude reply.
- `Default`: If none of the above conditions are met, it returns a default response asking the user to clarify or rephrase.

####  Defining the `chat` Function

In [ ]:
def chat():
    print("Chatbot: Hello! I am your support assistant. Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye! Have a great day!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

- This function handles the interaction loop with the user.
 - The chatbot initially greets the user with a welcome message and explains that typing "exit" will end the conversation.
 - The `while` True loop keeps the conversation going until the user types `'exit'`.
 - The user's input is taken using `input()`.
 - If the user types `"exit"`, the chatbot says goodbye and the loop is terminated using `break`.
 - If the input is anything other than "exit", it passes the input to the `chatbot_response()` function to generate a response and then prints the chatbot's response.

#### Running the Chatbot

In [ ]:
if __name__ == "__main__":
    chat()

Chatbot: Hello! I am your support assistant. Type 'exit' to end the chat.
You: 
Chatbot: I'm sorry, I didn't understand that. Can you please rephrase?
You: exit
Chatbot: Goodbye! Have a great day!


### Importing the Necessary Libraries:

In [ ]:
from transformers import pipeline

#### Loading the Pre-trained Model for Intent Recognition and Defining Possible Intents

In [ ]:
intent_recognizer = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
intents = ["greeting", "help", "thanks", "complaint", "feedback"]

#### Defining the Function for Intent Detection

In [ ]:
def detect_intent(user_input):
    result = intent_recognizer(user_input, intents)
    return result["labels"][0]  # Return the most likely intent

#### Enhanced Chatbot Response Function:

In [ ]:
def chatbot_response(user_input):
    intent = detect_intent(user_input)
    if intent == "greeting":
        return random.choice(responses["greeting"])
    elif intent == "help":
        return random.choice(responses["help"])
    elif intent == "thanks":
        return random.choice(responses["thanks"])
    elif intent == "complaint":
        return "I'm sorry to hear that. Can you please provide more details about the issue?"
    elif intent == "feedback":
        return "Thank you for your feedback! It helps us improve."
    else:
        return random.choice(responses["default"])

#### Running the Chatbot

In [ ]:
# Run chatbot
if __name__ == "__main__":
    chat()

Chatbot: Hello! I am your support assistant. Type 'exit' to end the chat.
You: exit
Chatbot: Goodbye! Have a great day!


#### Importing the Necessary Libraries:

In [ ]:
import random
import requests
from textblob import TextBlob
from transformers import pipeline
from googletrans import Translator
import logging
from datetime import datetime, timedelta

#### Logging Setup

In [ ]:
logging.basicConfig(filename="chatbot.log", level=logging.INFO)

#### Predefined Responses and Intent List

In [ ]:
responses = {
    "greeting": ["Hello {name}! How can I help you today?", "Hi {name}! What can I do for you?", "Greetings {name}! How may I assist you?"],
    "help": ["I'm here to help! What do you need assistance with?", "Sure, let me know how I can assist you.", "Feel free to ask me any question."],
    "thanks": ["You're welcome, {name}! Let me know if you need anything else.", "Glad I could help, {name}!", "Anytime, {name}!"],
    "complaint": ["I'm sorry to hear that, {name}. Can you please provide more details about the issue?"],
    "feedback": ["Thank you for your feedback! It helps us improve."],
    "default": ["I'm sorry, I didn't understand that. Can you please rephrase?", "Could you clarify your query? I'm here to help!"],
    "trivia": ["Did you know? Honey never spoils!", "Fun fact: Octopuses have three hearts.", "Here’s one: Bananas are berries, but strawberries aren’t!"],
    "jokes": ["Why don’t skeletons fight each other? They don’t have the guts!", "What do you call fake spaghetti? An impasta!", "Why don’t eggs tell jokes? They’d crack each other up!"],
    "reminder_set": ["Your reminder has been set for {time}! I'll remind you then."],
    "reminder_triggered": ["Hey {name}, here's your reminder: {reminder}"],
    "recommendation": {
        "book": ["I recommend 'The Alchemist' by Paulo Coelho!", "Try 'Atomic Habits' by James Clear.", "You might enjoy 'Sapiens' by Yuval Noah Harari."],
        "movie": ["How about watching 'Inception'?", "Try 'The Shawshank Redemption'—a classic!", "If you like animation, check out 'Spider-Man: Into the Spider-Verse'!"],
        "song": ["Listen to 'Blinding Lights' by The Weeknd.", "Try 'Shape of You' by Ed Sheeran.", "How about 'Bohemian Rhapsody' by Queen?"]
}
}

#### Intents List

In [ ]:
intents = ["greeting", "help", "thanks", "complaint", "feedback", "weather", "trivia", "jokes", "reminder", "recommendation"]

#### Conversation Context and Reminders

In [ ]:
context = {}
feedback_collected = []
reminders = {}

#### Intent Detection

In [ ]:
def detect_intent(user_input):
    result = intent_recognizer(user_input, intents)
    return result["labels"][0]  # Return the most likely intent

#### Sentiment Analysis

In [ ]:
def analyze_sentiment(user_input):
    sentiment = TextBlob(user_input).sentiment.polarity
    if sentiment > 0:
        return "positive"
    elif sentiment < 0:
        return "negative"
    return "neutral"

#### Weather Retrieval

In [ ]:
def get_weather(city):
    api_key = "yhttps://www.accuweather.com/en/in/bengaluru/204108/weather-forecast/204108"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url).json()
    if response.get("weather"):
        weather = response["weather"][0]["description"]
        temp = response["main"]["temp"] - 273.15  # Convert from Kelvin to Celsius
        return f"The weather in {city} is {weather} with a temperature of {temp:.2f}°C."
    return "Sorry, I couldn't fetch the weather data. Please try again later."

#### Translation Function

In [ ]:
def translate_message(message, target_language="en"):
    return translator.translate(message, dest=target_language).text

#### Reminder Functions

In [ ]:
def set_reminder(user_input, session_id):
    try:
        time_part = user_input.split("in ")[1]
        time_parts = time_part.split()
        minutes = int(time_parts[0])
        reminder_time = datetime.now() + timedelta(minutes=minutes)
        reminders[session_id] = (reminder_time, " ".join(time_parts[1:]))
        return responses["reminder_set"][0].format(time=reminder_time.strftime("%H:%M"))
    except Exception:
        return "I couldn't set the reminder. Please specify it in the format: 'Remind me in X minutes about Y'."

In [ ]:
# Check reminders
def check_reminders(session_id, name):
    if session_id in reminders:
        reminder_time, reminder_text = reminders[session_id]
        if datetime.now() >= reminder_time:
            del reminders[session_id]
            return responses["reminder_triggered"][0].format(name=name, reminder=reminder_text)
    return None

In [ ]:
# Collect feedback
def collect_feedback():
    feedback = input("Chatbot: Before you go, could you provide feedback on your experience? ")
    feedback_collected.append(feedback)
    print("Chatbot: Thank you for your feedback!")

In [ ]:
# Enhanced chatbot response
def chatbot_response(user_input, session_id="default", target_language="en"):
    if session_id not in context:
        name = input("Chatbot: What's your name? ")
        context[session_id] = {"name": name, "last_intent": None}
        print(f"Chatbot: Nice to meet you, {name}!")

    name = context[session_id]["name"]
    translated_input = translate_message(user_input, "en")
    intent = detect_intent(translated_input)
    sentiment = analyze_sentiment(translated_input)

    # Check reminders
    reminder_message = check_reminders(session_id, name)
    if reminder_message:
        return reminder_message

    logging.info(f"User Input: {user_input} | Detected Intent: {intent} | Sentiment: {sentiment}")

    # Intent-specific responses
    if intent == "greeting":
        response = random.choice(responses["greeting"]).format(name=name)
    elif intent == "help":
        response = random.choice(responses["help"])
    elif intent == "thanks":
        response = random.choice(responses["thanks"]).format(name=name)
    elif intent == "complaint":
        response = random.choice(responses["complaint"]).format(name=name)
    elif intent == "feedback":
        response = random.choice(responses["feedback"])
    elif intent == "weather":
        city = translated_input.split()[-1]
        response = get_weather(city)
    elif intent == "trivia":
        response = random.choice(responses["trivia"])
    elif intent == "jokes":
        response = random.choice(responses["jokes"])
    elif intent == "reminder":
        response = set_reminder(user_input, session_id)
    elif intent == "recommendation":
        recommendation_type = translated_input.split()[-1]
        response = random.choice(responses["recommendation"].get(recommendation_type, ["I'm not sure what to recommend."]))

    else:
        response = random.choice(responses["default"])

    return translate_message(response, target_language)

In [ ]:
# Chat loop
def chat():
    print("Chatbot: Hello! I am your support assistant. Type 'exit' to end the chat.")
    session_id = "default"
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            collect_feedback()
            print("Chatbot: Goodbye! Have a great day!")
            break
        response = chatbot_response(user_input, session_id)
        print(f"Chatbot: {response}")

In [ ]:
# Run chatbot
if __name__ == "__main__":
    chat()

Chatbot: Hello! I am your support assistant. Type 'exit' to end the chat.
Chatbot: Nice to meet you, Priyanka!
Chatbot: Hello Priyanka! How can I help you today?
You: exit
